In [ ]:
!pip install transformers datasets rouge-score

In [ ]:
pip install transformers[torch]

In [ ]:
pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer 

In [ ]:
model = SentenceTransformer

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples["article"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

training_args = TrainingArguments(output_dir="./results", evaluation_strategy="epoch", learning_rate=2e-5, per_device_train_batch_size=4, per_device_eval_batch_size=4, num_train_epochs=3)

trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset)
trainer.train()


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)

def evaluate_summaries(dataset):
    scores = []
    for article, highlights in zip(dataset["article"], dataset["highlights"]):
        # Generate summary here using your fine-tuned model
        generated_summary = generate_summary(article) # Replace with your summary generation function
        score = scorer.score(highlights, generated_summary)
        scores.append(score['rouge2'])
    return scores

# Example usage
test_scores = evaluate_summaries(test_dataset)
# Calculate average scores here
